In [356]:
import pandas as pd
import numpy as np

data = pd.read_csv('data/properatti.csv')
data.head(5)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,"3,430,787.00","-34.6618237,-58.5088387",-34.66,...,40.00,"1,127.27","1,550.00",nan,nan,nan,http://www.properati.com.ar/15bo8_venta_ph_mataderos_lavadero_patio_inmobiliaria-fedele,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO, REFACCIONADO A NUEVO, PATIO GRANDE, CON LAVADERO, LIVING COMEDOR CON COCINA INTEGRADA, ARTEFACTO DE COCINA, ALACENA, BAÑO COMPLETO, Y DORMITORIO. TODO EN EXCELENTE ESTADO, PARA HABITAR.NO ES APTO CREDITO Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIKufMJMwHtquzhk=/trim/198x0/smart/filters:strip_icc()/inmobiliariafedele.com/photos/CARHUE%202278-IMG_3400.JPG
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,"3,432,039.00","-34.9038831,-57.9643295",-34.90,...,nan,nan,nan,nan,nan,nan,http://www.properati.com.ar/15bob_venta_departamentos_la-plata_balcon_lavadero_toilette_garage_estrenar_antonini-propiedades_dcp,"Venta de departamento en décimo piso al frente, a estrenar. Living comedor con ventanales hacia el balcón con pisos de madera. Cocina completa con doble mesada, muebles bajo mesada. Lavadero separado. Toilette.Dormitorio con placard. Segundo dormitorio a dividir. Calefacción por radiadorBaño completo.Balcón corrido. Posibilidad de cochera semicubierta. U$D 20.000 Aviso publ...",VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv45KAKirGL3JM8=/trim/198x0/smart/filters:strip_icc()/antoninipropiedades.com/photos/401-1-7%20y%2037%2010A%201.jpg
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,"3,430,787.00","-34.6522615,-58.5229825",-34.65,...,55.00,"1,309.09","1,309.09",nan,nan,nan,http://www.properati.com.ar/15bod_venta_departamentos_mataderos_lavadero_placard_inmobiliaria-fedele,"2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AMPLIO COCINA SEPARADA CON LAVADERO, BAÑO COMPLETO Y DORMITORIO CON PLACARD. BAJAS EXPENSAS, EXCELENTE UBICACION Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).",2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_Ejwfu6QpW4cH0=/trim/198x0/smart/filters:strip_icc()/inmobiliariafedele.com/photos/GUAMINI%201171-IMG_3403.JPG
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,"3,431,333.00","-34.6477969,-58.5164244",-34.65,...,nan,nan,nan,nan,nan,nan,http://www.properati.com.ar/15boh_venta_ph_liniers_patio_g-goffredo,"PH 3 ambientes con patio. Hay 3 deptos en lote, 2 en PB y 1 en 1º piso.Reciclado a nuevo. ""Muy buen estado"" Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).",PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SPyleyeQlIypI8w=/trim/198x0/smart/filters:strip_icc()/ggoffredo.com/photos/GG0040-1563480102.jpg
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar del Plata|Centro|,Argentina,Buenos Aires Costa Atlántica,"3,435,548.00","-38.0026256,-57.5494468",-38.00,...,35.00,"1,828.57","1,828.57",nan,nan,nan,http://www.properati.com.ar/15bok_venta_departamentos_centro_cristina-pavone-negocios-inmobiliarios,"DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURAL, AL CONTRAFRENTE, EDIFICIO DE POCAS UNIDADES Y MUY BIEN CONSERVADO. EN ESTADO ORIGINAL.-POSEE: 1 DORMITORIOS, 1 BAÑO COMPLETO, LIVING COMEDOR Y COCINA SEPARADA.-EN ESTADO ORIGINAL.- Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).",DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA MITRE,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f5l8TrxdY-0kvs=/

In [357]:
''' Vemos las columnas '''

data.loc[10,:]


Unnamed: 0                                                                                                                                                                                                                                                                             10
operation                                                                                                                                                                                                                                                                            sell
property_type                                                                                                                                                                                                                                                                       house
place_name                                                                                                                                                

In [358]:
''' vemos los nulos '''

data['price_aprox_usd'].isna().sum()


20410

In [359]:
''' Exploramos las columnas de los precios para ver si hay datos no nulos'''

data.loc[data['price_aprox_usd'].isna()].loc[:,['price','price_aprox_local_currency','price_usd_per_m2','price_per_m2']]




,price,price_aprox_local_currency,price_usd_per_m2,price_per_m2
5,nan,nan,nan,nan
20,nan,nan,nan,nan
41,nan,nan,nan,nan
62,nan,nan,nan,nan
67,nan,nan,nan,nan
...,...,...,...,...
121210,nan,nan,nan,nan
121211,nan,nan,nan,nan
121212,nan,nan,nan,nan
121213,nan,nan,nan,nan


In [360]:
cant_nulls = is_null_result.sum()
print(cant_nulls)

Unnamed: 0                         0
operation                          0
property_type                      0
place_name                        23
place_with_parent_names            0
country_name                       0
state_name                         0
geonames_id                    18717
lat-lon                        51550
lat                            51550
lon                            51550
price                          20410
currency                       20411
price_aprox_local_currency     20410
price_aprox_usd                20410
surface_total_in_m2            39328
surface_covered_in_m2          19907
price_usd_per_m2               52603
price_per_m2                   33562
floor                         113321
rooms                          73830
expenses                      106958
properati_url                      0
description                        2
title                              0
image_thumbnail                 3112
dtype: int64


In [361]:
''' quitar los nulos precios '''

data = data.drop(data[data['price_aprox_usd'].isna()].index)
cant_nulls = data.isnull().sum()
print(cant_nulls)



Unnamed: 0                        0
operation                         0
property_type                     0
place_name                       23
place_with_parent_names           0
country_name                      0
state_name                        0
geonames_id                   16952
lat-lon                       42262
lat                           42262
lon                           42262
price                             0
currency                          1
price_aprox_local_currency        0
price_aprox_usd                   0
surface_total_in_m2           31944
surface_covered_in_m2         13150
price_usd_per_m2              32193
price_per_m2                  13152
floor                         93894
rooms                         61231
expenses                      87564
properati_url                     0
description                       2
title                             0
image_thumbnail                2250
dtype: int64


In [362]:
''' completar los places names '''
data.loc[data.place_name.isnull(),'place_name'] = data.loc[data.place_name.isnull()].place_with_parent_names.str.split('|', expand=True)[3]
cant_nulls = data.isnull().sum()
print(cant_nulls)


Unnamed: 0                        0
operation                         0
property_type                     0
place_name                        0
place_with_parent_names           0
country_name                      0
state_name                        0
geonames_id                   16952
lat-lon                       42262
lat                           42262
lon                           42262
price                             0
currency                          1
price_aprox_local_currency        0
price_aprox_usd                   0
surface_total_in_m2           31944
surface_covered_in_m2         13150
price_usd_per_m2              32193
price_per_m2                  13152
floor                         93894
rooms                         61231
expenses                      87564
properati_url                     0
description                       2
title                             0
image_thumbnail                2250
dtype: int64


In [363]:
cant_nulls = data.isnull().sum()
print(cant_nulls)

Unnamed: 0                        0
operation                         0
property_type                     0
place_name                        0
place_with_parent_names           0
country_name                      0
state_name                        0
geonames_id                   16952
lat-lon                       42262
lat                           42262
lon                           42262
price                             0
currency                          1
price_aprox_local_currency        0
price_aprox_usd                   0
surface_total_in_m2           31944
surface_covered_in_m2         13150
price_usd_per_m2              32193
price_per_m2                  13152
floor                         93894
rooms                         61231
expenses                      87564
properati_url                     0
description                       2
title                             0
image_thumbnail                2250
dtype: int64


In [364]:
''' Leo los ambientes para completar la info'''

import re

patron_ambientes = re.compile("(?P<Ambientes>\d+)(?P<tipo>amb|\samb)",flags = re.IGNORECASE)
patron_habitaciones = re.compile("(?P<Habitaciones>\d+)(?P<tipo>hab|\shab|dorm|\sdorm)",flags = re.IGNORECASE)

def extractRooms(row):
    ambientes_match = None
    if ( row['description'] and isinstance(row['description'],str) ):
        ambientes_match = patron_ambientes.search(row['description'])
    if ((ambientes_match is None) and row['title'] and isinstance(row['title'],str)):
        ambientes_match = patron_ambientes.search(row['title'])
    if (ambientes_match is not None):
        return int(ambientes_match.groups()[0]) 
    else: 
        if (row['description'] and isinstance(row['description'],str)):
            ambientes_match = patron_habitaciones.search(row['description'])
        if ((ambientes_match is None) and row['title'] and isinstance(row['title'],str)):
            ambientes_match = patron_habitaciones.search(row['title'])
        if (ambientes_match is not None):
            return int(ambientes_match.groups()[0])+1
        return None
        
        
data.loc[data.rooms.isnull(),'rooms'] = data.loc[data.rooms.isnull()].apply(extractRooms,axis=1)


In [365]:
cant_nulls = data.isnull().sum()
print(cant_nulls)

Unnamed: 0                        0
operation                         0
property_type                     0
place_name                        0
place_with_parent_names           0
country_name                      0
state_name                        0
geonames_id                   16952
lat-lon                       42262
lat                           42262
lon                           42262
price                             0
currency                          1
price_aprox_local_currency        0
price_aprox_usd                   0
surface_total_in_m2           31944
surface_covered_in_m2         13150
price_usd_per_m2              32193
price_per_m2                  13152
floor                         93894
rooms                         24521
expenses                      87564
properati_url                     0
description                       2
title                             0
image_thumbnail                2250
dtype: int64


In [371]:

patron_meters = re.compile("(?P<Metros>\d+)(?P<tipo>metro|\smetro|mts|\smts|m2|\sm2)",flags = re.IGNORECASE)


def extractMeters(row):
    meters_match = None
    if ( row['description'] and isinstance(row['description'],str) ):
        meters_match = patron_meters.search(row['description'])
    if ((patron_meters is None) and row['title'] and isinstance(row['title'],str)):
        meters_match = patron_meters.search(row['title'])
    if (meters_match is not None):
        return meters_match.groups()[0]
    return None

data.loc[data.surface_total_in_m2.isnull(),'surface_total_in_m2'] = data.loc[data.surface_total_in_m2.isnull()].apply(extractMeters,axis=1)

data.loc[data.surface_total_in_m2.isnull() & data.surface_covered_in_m2.notnull(),'surface_total_in_m2'] = data.loc[data.surface_total_in_m2.isnull() & data.surface_covered_in_m2.notnull()].apply(lambda x: x['surface_covered_in_m2'],axis=1)




In [379]:
cant_nulls = data.isnull().sum()
print(cant_nulls)

Unnamed: 0                        0
operation                         0
property_type                     0
place_name                        0
place_with_parent_names           0
country_name                      0
state_name                        0
geonames_id                   16952
lat-lon                       42262
lat                           42262
lon                           42262
price                             0
currency                          1
price_aprox_local_currency        0
price_aprox_usd                   0
surface_total_in_m2            9583
surface_covered_in_m2         11315
price_usd_per_m2              32193
price_per_m2                  13152
floor                         93894
rooms                         24521
expenses                      87564
properati_url                     0
description                       2
title                             0
image_thumbnail                2250
dtype: int64


In [408]:
pd.options.display.float_format = '{:,.2f}'.format

data.loc[data.surface_total_in_m2.isnull(),'surface_total_in_m2'] = data.loc[data.surface_total_in_m2.isnull()].surface_total_in_m2.fillna(value=np.nan)
data.loc[:,'surface_total_in_m2'] = data.loc[:,'surface_total_in_m2'].astype("float")




In [413]:

data.groupby(['state_name','property_type']).count()

Unnamed: 0  operation  place_name  \
state_name               property_type                                      
Bs.As. G.B.A. Zona Norte PH                    978        978         978   
                         apartment            9936       9936        9936   
                         house               12600      12600       12600   
                         store                 544        544         544   
Bs.As. G.B.A. Zona Oeste PH                    757        757         757   
...                                            ...        ...         ...   
Tierra Del Fuego         store                   3          3           3   
Tucumán                  PH                      2          2           2   
                         apartment             140        140         140   
                         house                  61         61          61   
                         store                  10         10          10   

                                        place_with_parent_names  country_name  \
state_name               property_type                                          
Bs.As. G.B.A. Zona Norte PH                                 978           978   
                         apartment                         9936          9936   
                         house                            12600         12600   
                         store                              544           544   
Bs.As. G.B.A. Zona Oeste PH                                 757           757   
...                                                         ...           ...   
Tierra Del Fuego         store                                3             3   
Tucumán                  PH                                   2             2   
                         apartment                          140           140   
                         house                               61            61   
                         store                               10            10   

                                        geonames_id  lat-lon   lat   lon  \
state_name               property_type                                     
Bs.As. G.B.A. Zona Norte PH                     730      441   441   441   
                         apartment             5332     4569  4569  4569   
                         house                 8127     5282  5282  5282   
                         store                  405      173   173   173   
Bs.As. G.B.A. Zona Oeste PH                     688      453   453   453   
...                                             ...      ...   ...   ...   
Tierra Del Fuego         store                    3        3     3     3   
Tucumán                  PH                       2        1     1     1   
                         apartment              112       17    17    17   
                         house                   44       13    13    13   
                         store                    7        1     1     1   

                                        price  ...  surface_covered_in_m2  \
state_name               property_type         ...                          
Bs.As. G.B.A. Zona Norte PH               978  ...                    907   
                         apartment       9936  ...                   9385   
                         house          12600  ...                  12028   
                         store            544  ...                    495   
Bs.As. G.B.A. Zona Oeste PH               757  ...                    634   
...                                       ...  ...                    ...   
Tierra Del Fuego         store              3  ...                      2   
Tucumán                  PH                 2  ...                      1   
                         apartment        140  ...                     93   
                         house             61  ...                     23   
                         store             10  ...                     